In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
%matplotlib inline

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from pathlib import Path

from tqdm.notebook import tqdm

In [ ]:
import os
os.environ['LOGURU_LEVEL'] = 'INFO'

In [ ]:
import logging

from loguru import logger

class InterceptHandler(logging.Handler):
    def emit(self, record):
        # Get corresponding Loguru level if it exists
        try:
            level = logger.level(record.levelname).name
        except ValueError:
            level = record.levelno

        # Find caller from where originated the logged message
        frame, depth = logging.currentframe(), 2
        while frame.f_code.co_filename == logging.__file__:
            frame = frame.f_back
            depth += 1

        logger.opt(depth=depth, exception=record.exc_info).log(level, record.getMessage())

logging.basicConfig(handlers=[InterceptHandler()], level=0)

In [ ]:
from datautils import generate_data

In [ ]:
%%time
# Train and val data
in_dir = Path('../../data/ICDAR2019_POCR_competition_dataset/ICDAR2019_POCR_competition_training_18M_without_Finnish')
data, md = generate_data(in_dir)

In [ ]:
# test data
in_dir = Path('../../data/ICDAR2019_POCR_competition_dataset/ICDAR2019_POCR_competition_evaluation_4M_without_Finnish')
data_test, md_test = generate_data(in_dir)

In [ ]:
out_dir = Path('icdar-dataset-20220207')

In [ ]:
X_train = pd.read_csv(out_dir/'train.csv', index_col=0)
X_val = pd.read_csv(out_dir/'val.csv', index_col=0)
X_test = pd.read_csv(out_dir/'test.csv', index_col=0)

In [ ]:
import dataclasses

from itertools import chain

tokens = []
val_files = set(X_val.file_name)

# Train and val
for key, d in tqdm(data.items()):
    for token in d.tokens:
        if token.ocr.strip() != token.gs.strip():
            r = dataclasses.asdict(token)
            r['language'] = key[:2]
            r['subset'] = key.split('/')[1]

            if key in val_files:
                r['dataset'] = 'val'
            else:
                r['dataset'] = 'train'

            tokens.append(r)
# Test
for key, d in tqdm(data_test.items()):
    for token in d.tokens:
        if token.ocr.strip() != token.gs.strip():
            r = dataclasses.asdict(token)
            r['language'] = key[:2]
            r['subset'] = key.split('/')[1]
            r['dataset'] = 'test'

            tokens.append(r)

In [ ]:
tdata = pd.DataFrame(tokens)

In [ ]:
print('token data:', tdata.shape[0], 'samples')

In [ ]:
tdata.dataset.value_counts()

In [ ]:
def update_data(tdata):
    tdata['ocr'] = tdata['ocr'].apply(lambda x: x.strip())
    tdata['gs'] = tdata['gs'].apply(lambda x: x.strip())
    tdata['len_ocr'] = tdata.apply(lambda row: len(row.ocr), axis=1)
    tdata['len_gs'] = tdata.apply(lambda row: len(row.gs), axis=1)
    tdata['diff'] = tdata.len_ocr - tdata.len_gs
    return tdata

tdata = update_data(tdata)

In [ ]:
tdata.len_ocr.describe()

In [ ]:
tdata.len_gs.describe()

In [ ]:
tdata['diff'].describe()

In [ ]:
tdata.len_ocr.hist(bins=1000, figsize=(10,5))

In [ ]:
tdata.len_ocr.hist(bins=1000, figsize=(10,5))
plt.ylim(0, 30)

In [ ]:
tdata.len_ocr.hist(bins=1000, figsize=(10,5))
plt.xlim(0, 30)

In [ ]:
from collections import Counter

sep = '<SEPSEPSEP>'

cs = {'train': Counter(), 'val': Counter(), 'test': Counter()}

for ocr, gs, ds in tqdm(zip(tdata.ocr.to_list(), 
                            tdata.gs.to_list(), 
                            tdata.dataset.to_list()), 
                            total=tdata.shape[0]):
    cs[ds][f'{ocr}{sep}{gs}'] += 1

In [ ]:
len(cs['train'])

In [ ]:
len(cs['val'])

In [ ]:
for k, v in cs['train'].most_common(25):
    ocr, gs = k.split(sep)
    print(repr(ocr), repr(gs), v)


In [ ]:
for k, v in cs['train'].items():
    if k.endswith(sep):
        print(k, v)

In [ ]:
# Only save unique ocr/gs pairs (for train and val)

def task2_dataset(c: Counter, sep: str) -> pd.DataFrame:
    samples = []
    for k, _ in c.items():
        ocr, gs = k.split(sep)
        samples.append({'ocr': ocr, 'gs': gs})
    return pd.DataFrame(samples)

train = task2_dataset(cs['train'], sep)
val = task2_dataset(cs['val'], sep)
test = task2_dataset(cs['test'], sep)

In [ ]:
train.to_csv(out_dir/'task2_train.csv')
val.to_csv(out_dir/'task2_val.csv')

In [ ]:
test.to_csv(out_dir/'task2_test.csv')

In [ ]:
train = pd.read_csv(out_dir/'task2_train.csv', index_col=0)
val = pd.read_csv(out_dir/'task2_val.csv', index_col=0)
test = pd.read_csv(out_dir/'task2_test.csv', index_col=0)

train = train.fillna('')
val = val.fillna('')
test = test.fillna('')